In [176]:
import os
from dotenv import load_dotenv

In [177]:
load_dotenv()

True

In [178]:
#os.getenv('OPENAI_API_KEY')

### OpenAI API

In [179]:
import json
from openai import OpenAI

In [180]:
client = OpenAI()

In [181]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    n=1,
    temperature=0.5,
    messages=[
        {
            "role": "user",
            "content": "Who won first cricket worldcup?",
        }
    ]
)
response.choices[0].message.content

'The first Cricket World Cup was won by the West Indies in 1975. They defeated Australia in the final to become the inaugural champions.'

Check tokens calculation [https://platform.openai.com/tokenizer](https://platform.openai.com/tokenizer)

Check Pricing for tokens [https://openai.com/api/pricing/](https://openai.com/api/pricing/)

### Function Calling
connect large language models to external tools

In [182]:
prompt = f'''
please extract the following information from the given text in JSON format.

year
winner_team
runner_up_team

Information:
First cricket worldcup was played in 1975 between West Indies and Australia. This worldcup was won by West Indies.
'''
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)
json.loads(response.choices[0].message.content)

{'year': 1975, 'winner_team': 'West Indies', 'runner_up_team': 'Australia'}

#### Same as above but using Function Calling

In [183]:
functions = [
    {
        "name": "extract_details",
        "parameters": {
            "type": "object",
            "properties": {
                "year": {"type": "integer"},
                "winner_team": {"type": "string"},
                "runner_up_team": {"type": "string"},
            }
        }
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    functions=functions,
    function_call="auto",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)
json.loads(response.choices[0].message.function_call.arguments)

{}

### Inline function call

In [184]:
from datetime import datetime, timedelta

In [185]:
prompt = "when is the next cricket match between pakistan and india?"

def get_next_match(team1: str, team2: str):
    # dummy function to return next match date between two teams. this could be replaced with espncricinfo api to get real data
    return json.dumps({"team1": team1, "team2": team2, "next_match_date": str(datetime.today() + timedelta(days=7))})

functions = [
    {
        "name": "get_next_match",
        "parameters": {
            "type": "object",
            "properties": {
                "team1": {"type": "string"},
                "team2": {"type": "string"},
                "next_match_date": {"type": "string"},
            },
            "required": ["team1", "team2"]
        }
    }
]

response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "user", "content": prompt}
    ]
)
params = json.loads(response1.choices[0].message.function_call.arguments)
get_next_match_func = eval(response1.choices[0].message.function_call.name)(**params)

response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "user", "content": prompt},
        {"role": "function", "name": response1.choices[0].message.function_call.name, "content": get_next_match_func}
    ]
)

response2.choices[0].message.content

'The next cricket match between Pakistan and India is scheduled to take place on June 23, 2024.'